In [25]:
import numpy as np
import pandas as pd

from itertools import chain
from datetime import datetime

from catboost import CatBoostRegressor

In [26]:
model = CatBoostRegressor()
model.load_model("./final-model.cbm")

In [101]:
df: pd.DataFrame = pd.read_pickle("./prepared-dataset.pkl")
df

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Median LF Кабина,IQR LF Кабина,Min Бронирования по кабинам,Max Бронирования по кабинам,Median Бронирования по кабинам,IQR Бронирования по кабинам,Min Пассажиры,Max Пассажиры,Median Пассажиры,IQR Пассажиры
0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,32B,...,1.0539,0.04790,150.96,182.26,176.0,8.0,151.74,184.26,176.0,8.00
1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,333,...,0.3214,0.10710,3.32,14.00,9.0,3.0,3.32,15.68,9.0,4.00
2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,77W,...,0.5387,0.31200,69.08,281.00,202.0,117.0,69.56,290.44,213.0,109.00
3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,77W,...,0.5000,0.41660,1.00,21.94,12.0,10.0,1.00,21.94,12.5,10.00
4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,77W,...,0.4286,0.39280,2.04,26.92,12.0,11.0,2.04,26.92,12.0,11.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,32B,...,0.7586,0.53445,1.00,29.00,22.0,15.5,1.00,30.20,22.0,16.00
19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,32B,...,1.1014,0.08700,136.82,171.00,152.0,12.0,136.82,171.00,153.0,13.00
19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,32B,...,0.6875,0.31250,4.00,16.00,11.0,5.0,5.00,16.00,11.0,5.00
19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,32B,...,1.0507,0.08700,131.00,163.54,145.0,12.0,131.00,162.54,145.0,12.00


In [102]:
TARGET_FEATURE = "Доход пасс"

CAT_FEATURES = ["Аэропорт вылета", "Аэропорт прилета", "Тип ВС",
                "Город аэропорта вылета", "Город аэропорта прилета",
                "Страна аэропорта вылета", "Страна аэропорта прилета",
                "Часовой пояс аэропорта вылета", "Часовой пояс аэропорта прилета",
                "Код кабины"]

EXCLUDE_FEATURES = [x for x in list(chain.from_iterable([
    [column for column in df.columns if column.startswith(prefix)]
    for prefix in [
        "Доход"
    ]
])) if x != "Доход пасс"]

In [103]:
def combine_date_minutes(date_val, minutes):
    if pd.isna(date_val):
        return pd.NaT
    dt_date = pd.to_datetime(date_val).normalize()
    return dt_date + pd.to_timedelta(int(minutes), unit='m')

In [104]:
df = df.drop(EXCLUDE_FEATURES, axis=1)

df['Время вылета_ts'] = df.apply(lambda r: combine_date_minutes(r['Дата вылета'], r['Время вылета (минуты)']), axis=1)
df['Дата вылета'] = pd.to_datetime(df['Дата вылета'])

df = df.reset_index().rename(columns={'index': 'flight_instance_id'})

df

,flight_instance_id,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,...,IQR LF Кабина,Min Бронирования по кабинам,Max Бронирования по кабинам,Median Бронирования по кабинам,IQR Бронирования по кабинам,Min Пассажиры,Max Пассажиры,Median Пассажиры,IQR Пассажиры,Время вылета_ts
0,0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,...,0.04790,150.96,182.26,176.0,8.0,151.74,184.26,176.0,8.00,2024-08-01 10:00:00
1,1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,...,0.10710,3.32,14.00,9.0,3.0,3.32,15.68,9.0,4.00,2024-08-01 16:30:00
2,2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,...,0.31200,69.08,281.00,202.0,117.0,69.56,290.44,213.0,109.00,2024-08-01 10:35:00
3,3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,...,0.41660,1.00,21.94,12.0,10.0,1.00,21.94,12.5,10.00,2024-08-01 10:35:00
4,4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,...,0.39280,2.04,26.92,12.0,11.0,2.04,26.92,12.0,11.00,2024-08-01 10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,...,0.53445,1.00,29.00,22.0,15.5,1.00,30.20,22.0,16.00,2025-06-02 03:00:00
19265,19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,...,0.08700,136.82,171.00,152.0,12.0,136.82,171.00,153.0,13.00,2025-06-02 03:00:00
19266,19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,...,0.31250,4.00,16.00,11.0,5.0,5.00,16.00,11.0,5.00,2025-06-02 11:50:00
19267,19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,...,0.08700,131.00,163.54,145.0,12.0,131.00,162.54,145.0,12.00,2025-06-02 11:50:00


In [105]:
def prepare_data(df):
    df = df.copy()
    df["flight_id"] = (
        df["Номер рейса"].astype(str) + "_" +
        df["Аэропорт вылета"].astype(str) + "_" +
        df["Дата вылета"].dt.strftime("%Y-%m-%d") + "_" +
        df["Время вылета"].astype(str)
    )
    return df

df = prepare_data(df)
df

,flight_instance_id,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,...,Min Бронирования по кабинам,Max Бронирования по кабинам,Median Бронирования по кабинам,IQR Бронирования по кабинам,Min Пассажиры,Max Пассажиры,Median Пассажиры,IQR Пассажиры,Время вылета_ts,flight_id
0,0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,...,150.96,182.26,176.0,8.0,151.74,184.26,176.0,8.00,2024-08-01 10:00:00,1479_ABA_2024-08-01_10:00:00
1,1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,...,3.32,14.00,9.0,3.0,3.32,15.68,9.0,4.00,2024-08-01 16:30:00,427_SSH_2024-08-01_16:30:00
2,2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,...,69.08,281.00,202.0,117.0,69.56,290.44,213.0,109.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00
3,3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,...,1.00,21.94,12.0,10.0,1.00,21.94,12.5,10.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00
4,4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,...,2.04,26.92,12.0,11.0,2.04,26.92,12.0,11.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,19264,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,...,1.00,29.00,22.0,15.5,1.00,30.20,22.0,16.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00
19265,19265,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,...,136.82,171.00,152.0,12.0,136.82,171.00,153.0,13.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00
19266,19266,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,...,4.00,16.00,11.0,5.0,5.00,16.00,11.0,5.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00
19267,19267,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,...,131.00,163.54,145.0,12.0,131.00,162.54,145.0,12.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00


In [33]:
class FrequencyConstraints:
    def __init__(self, route_col: str = 'Номер рейса'):
        self.route_col = route_col
        self.min_requirements = {}
    
    @staticmethod
    def interval_key_for_date(dt: pd.Timestamp, interval: str) -> str:
        if interval == 'week':
            return f"{dt.year}-W{dt.isocalendar()[1]}"
        else:
            raise ValueError('unsupported interval')

    def set_min(self, route, interval_label, min_count):
        self.min_requirements[(route, interval_label)] = int(min_count)

    def get_min(self, route, interval_label):
        return self.min_requirements.get((route, interval_label), 0)

    def compute_counts(self, df_schedule: pd.DataFrame, time_col_ts: str = 'Время вылета_ts'):
        counts = {}
        for i, row in df_schedule.iterrows():
            route = row[self.route_col]
            dt = pd.Timestamp(row[time_col_ts])
            ik = self.interval_key_for_date(dt, 'week')
            label = f"week:{ik}"
            counts[(route, label)] = counts.get((route, label), 0) + 1
        return counts

In [34]:
def build_weekly_constraints(df: pd.DataFrame, time_col_ts="Время вылета_ts", route_col="Номер рейса"):
    constraints = FrequencyConstraints(route_col=route_col)
    for _, row in df.iterrows():
        route = row[route_col]
        dt = pd.Timestamp(row[time_col_ts])
        label = f"week:{FrequencyConstraints.interval_key_for_date(dt, 'week')}"
        constraints.min_requirements[(route, label)] = constraints.min_requirements.get((route, label), 0) + 1
    return constraints

In [80]:
import hashlib

def dt_to_slot(dt: pd.Timestamp, slot_minutes: int = 60) -> int:
    if pd.isna(dt):
        return None
    ts = int(dt.value // 10**9)
    minutes = ts // 60
    return minutes // slot_minutes

def slot_to_dt(slot: int, slot_minutes: int = 60) -> pd.Timestamp:
    minutes = slot * slot_minutes
    secs = minutes * 60
    return pd.Timestamp(secs, unit='s')

def hash_key(*args) -> str:
    m = hashlib.sha1()
    for a in args:
        m.update(str(a).encode())
        m.update(b"|")
    return m.hexdigest()

class Predictor:
    def __init__(self, model: CatBoostRegressor, slot_minutes: int = 60):
        self.model = model
        self.cache = {}
        self.slot_minutes = slot_minutes
        self.feature_cols = model.feature_names_

    def predict_batch(self, df_features: pd.DataFrame) -> np.ndarray:
        X = df_features[self.feature_cols].copy()
        for col in X.columns:
            if pd.api.types.is_object_dtype(X[col]):
                if isinstance(X[col].iloc[0], datetime) or isinstance(X[col].iloc[0], pd.Timestamp):
                    X[col] = pd.to_datetime(X[col])
        return self.model.predict(X)

    def predict_for_schedule(self, df_schedule: pd.DataFrame, time_col_ts: str = 'Время вылета_ts') -> np.ndarray:
        preds = np.zeros(len(df_schedule), dtype=float)
        to_compute_idx = []
        for i, row in df_schedule.iterrows():
            fid = row.get('flight_instance_id', i)
            slot = dt_to_slot(pd.Timestamp(row[time_col_ts]), slot_minutes=self.slot_minutes)
            key = hash_key(fid, slot)
            if key in self.cache:
                preds[i] = self.cache[key]
            else:
                to_compute_idx.append(i)
        if to_compute_idx:
            batch = df_schedule.loc[to_compute_idx].copy()
            batch_preds = self.predict_batch(batch)
            for j, pred in enumerate(batch_preds):
                idx = to_compute_idx[j]
                fid = batch.iloc[j].get('flight_instance_id', idx)
                slot = dt_to_slot(pd.Timestamp(batch.iloc[j][time_col_ts]), slot_minutes=self.slot_minutes)
                key = hash_key(fid, slot)
                self.cache[key] = pred
                preds[idx] = pred
        return preds

In [81]:
def operator_shift(df: pd.DataFrame, flight_id: str, shift_slots: int,
                          time_col_ts: str = 'Время вылета_ts', slot_minutes: int = 60) -> pd.DataFrame:
    df2 = df.copy()
    mask = df2['flight_id'] == flight_id
    old_departure = pd.Timestamp(df2.loc[mask, time_col_ts].iloc[0])
    old_slot = dt_to_slot(old_departure, slot_minutes=slot_minutes)
    new_slot = old_slot + shift_slots
    new_departure = slot_to_dt(new_slot, slot_minutes=slot_minutes)
    
    # сохраняем продолжительность рейса
    duration = pd.Timedelta(minutes=int(df2.loc[mask, 'Время в пути'].iloc[0]))
    
    df2.loc[mask, time_col_ts] = new_departure
    df2.loc[mask, 'Дата вылета'] = new_departure
    df2.loc[mask, 'Время вылета (минуты)'] = int((new_departure - pd.Timestamp(new_departure.date())).total_seconds() // 60)
    
    new_arrival = new_departure + duration
    df2.loc[mask, 'Время прилета'] = new_arrival.time()
    df2.loc[mask, 'Время прилета (минуты)'] = int((new_arrival - pd.Timestamp(new_arrival.date())).total_seconds() // 60)
    
    return df2

def operator_swap(df: pd.DataFrame, flight_id1: str, flight_id2: str,
                          time_col_ts: str = 'Время вылета_ts') -> pd.DataFrame:
    df2 = df.copy()
    mask1 = df2['flight_id'] == flight_id1
    mask2 = df2['flight_id'] == flight_id2

    t1 = pd.Timestamp(df2.loc[mask1, time_col_ts].iloc[0])
    t2 = pd.Timestamp(df2.loc[mask2, time_col_ts].iloc[0])

    df2.loc[mask1, time_col_ts] = t2
    df2.loc[mask2, time_col_ts] = t1
    
    for mask in [mask1, mask2]:
        new_departure = pd.Timestamp(df2.loc[mask, time_col_ts].iloc[0])
        duration = pd.Timedelta(minutes=int(df2.loc[mask, 'Время в пути'].iloc[0]))
        df2.loc[mask, 'Дата вылета'] = new_departure
        df2.loc[mask, 'Время вылета (минуты)'] = int((new_departure - pd.Timestamp(new_departure.date())).total_seconds() // 60)
        new_arrival = new_departure + duration
        df2.loc[mask, 'Время прилета'] = new_arrival.time()
        df2.loc[mask, 'Время прилета (минуты)'] = int((new_arrival - pd.Timestamp(new_arrival.date())).total_seconds() // 60)
    
    return df2

In [82]:
def repair_schedule(df: pd.DataFrame,
                           freq_constraints: 'FrequencyConstraints',
                           time_col_ts: str = 'Время вылета_ts',
                           max_shift_slots: int = 24,
                           slot_minutes: int = 60):
    df2 = df.copy()
    counts = freq_constraints.compute_counts(df2, time_col_ts=time_col_ts)
    flight_to_indices = df2.groupby('flight_id').indices

    for (route, interval_label), minreq in freq_constraints.min_requirements.items():
        current = counts.get((route, interval_label), 0)
        if current >= minreq:
            continue
        deficit = minreq - current
        _, ik = interval_label.split(':', 1)
        year, week = map(int, ik.split('-W'))
        target_dt = pd.Timestamp.fromisocalendar(year, week, 4)  # Thursday
        target_slot = dt_to_slot(target_dt, slot_minutes=slot_minutes)

        moved_count = 0
        for fid in df2[df2[freq_constraints.route_col] == route]['flight_id'].unique():
            first_idx = flight_to_indices[fid][0]
            src_dt = pd.Timestamp(df2.at[first_idx, time_col_ts])
            src_slot = dt_to_slot(src_dt, slot_minutes=slot_minutes)
            shift = target_slot - src_slot
            if abs(shift) <= max_shift_slots:
                df2 = operator_shift(df2, fid, shift, time_col_ts=time_col_ts, slot_minutes=slot_minutes)
                moved_count += 1
                if moved_count >= deficit:
                    break
        if moved_count < deficit:
            return df2, False
    return df2, True

In [ ]:
def local_search_improved(df_initial: pd.DataFrame,
                                 predictor: Predictor,
                                 freq_constraints: 'FrequencyConstraints',
                                 time_col_ts: str = 'Время вылета_ts',
                                 slot_minutes: int = 60,
                                 max_iter: int = 3000,
                                 max_shift_slots: int = 24,
                                 accept_worse_prob: float = 0.05,
                                 decay_rate: float = 0.999) -> pd.DataFrame:
    df = df_initial.copy()
    df['was_moved'] = False
    df['_pred'] = df['Доход пасс']

    # --- базовый доход (по каждой кабине, суммарно по flight_id)
    flight_base_scores = df.groupby('flight_id')["Доход пасс"].sum().to_dict()

    # --- initial score берём из фактического дохода
    best_score = df["Доход пасс"].sum()
    best_df = df.copy()
    print(f"initial score = {best_score:.2f}")

    rng = np.random.default_rng()

    for it in range(max_iter):
        step_slots = max(1, int(max_shift_slots * decay_rate ** it))

        # выбираем flight_id для изменения
        flight_ids = df['flight_id'].unique()
        if rng.random() < 0.7:
            # 70% шанс брать из слабых (по суммарному прогнозу)
            flight_scores = df.groupby('flight_id')['_pred'].sum()
            weakest_ids = flight_scores.nsmallest(max(1, len(flight_ids)//3)).index
            fid = rng.choice(weakest_ids)
        else:
            fid = rng.choice(flight_ids)

        # выбираем оператор
        op = rng.choice(['shift', 'swap'], p=[0.7, 0.3])

        if op == 'shift':
            shift = int(rng.integers(-step_slots, step_slots + 1))
            if shift == 0:
                continue
            cand = operator_shift(df, fid, shift, time_col_ts=time_col_ts, slot_minutes=slot_minutes)
            cand.loc[cand['flight_id'] == fid, 'was_moved'] = True
        else:
            # swap с другим случайным рейсом
            other_fid = rng.choice(flight_ids)
            if other_fid == fid:
                continue
            cand = operator_swap(df, fid, other_fid, time_col_ts=time_col_ts)
            cand.loc[cand['flight_id'] == fid, 'was_moved'] = True
            cand.loc[cand['flight_id'] == other_fid, 'was_moved'] = True

        # repair_schedule_flight с freq_constraints и slot_constraints
        repaired, ok = repair_schedule(cand, freq_constraints,
                                              time_col_ts=time_col_ts, max_shift_slots=max_shift_slots,
                                              slot_minutes=slot_minutes)
        if not ok:
            print(f"iter {it}: failed constraints")
            continue  # пропускаем кандидат

        # пересчёт прогнозов
        cand_preds = predictor.predict_for_schedule(repaired, time_col_ts=time_col_ts)
        repaired['_pred'] = cand_preds

        # проверка: доход каждого flight_id не ниже исходного
        valid = True
        for fid_check, group in repaired.groupby('flight_id'):
            if not repaired.loc[group.index, 'was_moved'].any():
                # если полёт ещё не сдвигался, берем исходный доход
                current_sum = flight_base_scores[fid_check]
            else:
                current_sum = group['_pred'].sum()
            if current_sum < flight_base_scores[fid_check]:
                valid = False
                break
        if not valid:
            print(f"iter {it}: invalid sum")
            continue

        cand_score = repaired['_pred'].sum()
        prob = accept_worse_prob * (decay_rate ** it)

        if cand_score > best_score or rng.random() < prob:
            df = repaired
            if cand_score > best_score:
                best_df = repaired.copy()
                best_score = cand_score
                print(f"iter {it}: improved score = {best_score:.2f}")
            else:
                print(f"iter {it}: failed score")

    # --- итоговый доход
    best_df['Доход пасс'] = best_df.apply(
        lambda row: row['_pred'] if row['was_moved'] else row['Доход пасс'],
        axis=1
    )

    best_df.drop(columns=['_pred'], inplace=True, errors='ignore')

    return best_df


In [180]:
SLOT_MINUTES = 15

predictor = Predictor(model=model, slot_minutes=SLOT_MINUTES)
constraints = build_weekly_constraints(df, time_col_ts='Время вылета_ts', route_col='Номер рейса')

In [181]:
optimized = local_search_improved(
    df, predictor,
    freq_constraints=constraints,
    time_col_ts='Время вылета_ts',
    slot_minutes=SLOT_MINUTES, max_iter=1000,
    # час * до 24 часов
    max_shift_slots=int((60/SLOT_MINUTES)*24*3)
)

initial score = 233571300.94
iter 0: improved score = 264520968.40
iter 1: improved score = 264762524.91
iter 2: improved score = 265042203.98
iter 3: improved score = 265271841.56
iter 4: invalid sum
iter 5: improved score = 265769739.33
iter 6: improved score = 266031186.79
iter 7: invalid sum
iter 8: improved score = 266544429.29
iter 9: improved score = 266827823.48
iter 10: improved score = 267118958.38
iter 11: invalid sum
iter 12: improved score = 267684543.75
iter 13: invalid sum
iter 14: improved score = 268230923.01
iter 15: invalid sum
iter 16: invalid sum
iter 17: improved score = 269079270.91
iter 18: failed constraints
iter 19: improved score = 269652868.03
iter 20: improved score = 269943428.81
iter 21: improved score = 270239319.57
iter 22: improved score = 270538067.15
iter 23: invalid sum
iter 24: improved score = 271193259.02
iter 25: invalid sum
iter 26: improved score = 271816707.22
iter 27: improved score = 272133218.02
iter 28: improved score = 272447919.35
iter 

In [182]:
optimized

,flight_instance_id,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,...,Max Бронирования по кабинам,Median Бронирования по кабинам,IQR Бронирования по кабинам,Min Пассажиры,Max Пассажиры,Median Пассажиры,IQR Пассажиры,Время вылета_ts,flight_id,was_moved
0,0,2024-08-01 00:00:00,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,...,182.26,176.0,8.0,151.74,184.26,176.0,8.00,2024-08-01 10:00:00,1479_ABA_2024-08-01_10:00:00,False
1,1,2024-08-03 08:30:00,427,SSH,SVO,16:30:00,14:30:00,28,0.1071,3,...,14.00,9.0,3.0,3.32,15.68,9.0,4.00,2024-08-03 08:30:00,427_SSH_2024-08-01_16:30:00,True
2,2,2024-08-01 00:00:00,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,...,281.00,202.0,117.0,69.56,290.44,213.0,109.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False
3,3,2024-08-01 00:00:00,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,...,21.94,12.0,10.0,1.00,21.94,12.5,10.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False
4,4,2024-08-01 00:00:00,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,...,26.92,12.0,11.0,2.04,26.92,12.0,11.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19264,19264,2025-06-02 00:00:00,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,...,29.00,22.0,15.5,1.00,30.20,22.0,16.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00,False
19265,19265,2025-06-02 00:00:00,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,...,171.00,152.0,12.0,136.82,171.00,153.0,13.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00,False
19266,19266,2025-06-02 00:00:00,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,...,16.00,11.0,5.0,5.00,16.00,11.0,5.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00,False
19267,19267,2025-06-02 00:00:00,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,...,163.54,145.0,12.0,131.00,162.54,145.0,12.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00,False


In [183]:
result = optimized.copy()   
result[result['was_moved'] == True][['Время вылета_ts', "flight_id", "Код кабины"]]

,Время вылета_ts,flight_id,Код кабины
1,2024-08-03 08:30:00,427_SSH_2024-08-01_16:30:00,C
7,2024-10-18 15:30:00,1524_SVO_2024-08-01_01:05:00,C
8,2024-10-18 15:30:00,1524_SVO_2024-08-01_01:05:00,Y
21,2025-04-15 16:40:00,6574_KJA_2024-08-01_06:55:00,C
22,2025-04-15 16:40:00,6574_KJA_2024-08-01_06:55:00,Y
...,...,...,...
18749,2025-02-19 14:40:00,1009_KGD_2025-05-24_03:00:00,Y
18841,2025-05-25 11:15:00,1525_RGK_2025-05-26_11:05:00,Y
18842,2025-05-25 11:15:00,1525_RGK_2025-05-26_11:05:00,C
19008,2025-05-18 16:25:00,1055_MCX_2025-05-29_06:20:00,Y


In [186]:
df_initial = df.copy()
df_initial['was_moved'] = False

df_initial['stage'] = 'start'
result['stage'] = 'result'

df_combined = pd.concat([df_initial, result], ignore_index=True)
df_combined['Дата вылета'] = pd.to_datetime(df_combined['Время вылета_ts'].dt.date)
df_combined['Время вылета'] = df_combined['Время вылета_ts'].dt.time
df_combined['Время вылета (минуты)'] = df_combined['Время вылета_ts'].dt.hour * 60 + df_combined['Время вылета_ts'].dt.minute
df_combined = df_combined.drop(columns=['flight_instance_id'])
df_combined

,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Емкость кабины,LF Кабина,Бронирования по кабинам,Тип ВС,...,Median Бронирования по кабинам,IQR Бронирования по кабинам,Min Пассажиры,Max Пассажиры,Median Пассажиры,IQR Пассажиры,Время вылета_ts,flight_id,was_moved,stage
0,2024-08-01,1479,ABA,SVO,10:00:00,11:00:00,167,1.0299,172,32B,...,176.0,8.0,151.74,184.26,176.0,8.00,2024-08-01 10:00:00,1479_ABA_2024-08-01_10:00:00,False,start
1,2024-08-01,427,SSH,SVO,16:30:00,22:30:00,28,0.1071,3,333,...,9.0,3.0,3.32,15.68,9.0,4.00,2024-08-01 16:30:00,427_SSH_2024-08-01_16:30:00,False,start
2,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,375,0.5467,205,77W,...,202.0,117.0,69.56,290.44,213.0,109.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False,start
3,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,24,0.2500,6,77W,...,12.0,10.0,1.00,21.94,12.5,10.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False,start
4,2024-08-01,275,HKT,SVO,10:35:00,16:40:00,28,0.2500,7,77W,...,12.0,11.0,2.04,26.92,12.0,11.00,2024-08-01 10:35:00,275_HKT_2024-08-01_10:35:00,False,start
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38533,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,29,0.5517,16,32B,...,22.0,15.5,1.00,30.20,22.0,16.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00,False,result
38534,2025-06-02,1009,KGD,SVO,03:00:00,06:30:00,138,1.1014,152,32B,...,152.0,12.0,136.82,171.00,153.0,13.00,2025-06-02 03:00:00,1009_KGD_2025-06-02_03:00:00,False,result
38535,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,16,0.8750,14,32B,...,11.0,5.0,5.00,16.00,11.0,5.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00,False,result
38536,2025-06-02,1018,SVO,KGD,11:50:00,13:35:00,138,1.0290,142,32B,...,145.0,12.0,131.00,162.54,145.0,12.00,2025-06-02 11:50:00,1018_SVO_2025-06-02_11:50:00,False,result


In [ ]:
df_combined.to_pickle("frontend.pkl")

In [ ]:
frontend = df_combined.copy()
frontend = frontend[['Дата вылета', "Номер рейса", "Аэропорт вылета", "Аэропорт прилета",
               "Время вылета", "Время прилета", "Тип ВС", "Доход пасс", "Код кабины",
               "Емкость кабины", "Пассажиры", "Бронирования по кабинам",
               "Широта аэропорта вылета", "Долгота аэропорта вылета",
               "Широта аэропорта прилета", "Долгота аэропорта прилета",
               "Город аэропорта вылета", "Город аэропорта прилета",
               "Страна аэропорта вылета", "Страна аэропорта прилета",
               'Расстояние между аэропортами', 'Международный рейс', 'Вектор движения',
               'Популяция аэропорта вылета', 'Часовой пояс аэропорта вылета',
               'Популяция аэропорта прилета', 'Часовой пояс аэропорта прилета',
               'Время в пути', "Время вылета_ts", 'flight_id', 'was_moved', 'stage',
               "Время вылета (минуты)", "Время прилета (минуты)"
              ]]

# группируем по flight_id
grouping_columns = ['flight_id', 'stage']
result_rows = []

for fid, group in frontend.groupby(grouping_columns):
    # выбираем одно значение для общих характеристик рейса (берем первый, т.к. они одинаковые для всех кабин)
    row_base = group.iloc[0]

    # агрегаты по рейсу
    total_revenue = group['Доход пасс'].sum()
    total_capacity = group['Емкость кабины'].sum()
    total_passengers = group['Пассажиры'].sum()
    total_bookings = group['Бронирования по кабинам'].sum()

    row_data = {
        "flight_id": row_base['flight_id'],
        "Дата вылета": row_base['Дата вылета'],
        "Номер рейса": row_base['Номер рейса'],
        "Аэропорт вылета": row_base['Аэропорт вылета'],
        "Аэропорт прилета": row_base['Аэропорт прилета'],
        "Время вылета": row_base['Время вылета'],
        "Время прилета": row_base['Время прилета'],
        "Время вылета (минуты)": row_base['Время вылета (минуты)'],
        "Время прилета (минуты)": row_base['Время прилета (минуты)'],
        "Тип ВС": row_base['Тип ВС'],
        "Широта аэропорта вылета": row_base['Широта аэропорта вылета'],
        "Долгота аэропорта вылета": row_base['Долгота аэропорта вылета'],
        "Широта аэропорта прилета": row_base['Широта аэропорта прилета'],
        "Долгота аэропорта прилета": row_base['Долгота аэропорта прилета'],
        "Город аэропорта вылета": row_base['Город аэропорта вылета'],
        "Город аэропорта прилета": row_base['Город аэропорта прилета'],
        "Страна аэропорта вылета": row_base['Страна аэропорта вылета'],
        "Страна аэропорта прилета": row_base['Страна аэропорта прилета'],
        "Расстояние между аэропортами": row_base['Расстояние между аэропортами'],
        "Международный рейс": row_base['Международный рейс'],
        "Вектор движения": row_base['Вектор движения'],
        "Популяция аэропорта вылета": row_base['Популяция аэропорта вылета'],
        "Часовой пояс аэропорта вылета": row_base['Часовой пояс аэропорта вылета'],
        "Популяция аэропорта прилета": row_base['Популяция аэропорта прилета'],
        "Часовой пояс аэропорта прилета": row_base['Часовой пояс аэропорта прилета'],
        "Время в пути": row_base['Время в пути'],
        "Время вылета_ts": row_base['Время вылета_ts'],
        "was_moved": row_base['was_moved'],
        "stage": row_base['stage'],
        # агрегаты
        "Доход": total_revenue,
        "Емкость": total_capacity,
        "Пассажиры": total_passengers,
        "Бронирования": total_bookings,
        "Загруженность": total_bookings / total_capacity if total_capacity > 0 else 0,
        "Доход на пассажира": total_revenue / total_passengers if total_passengers > 0 else 0
    }

    result_rows.append(row_data)

frontend = pd.DataFrame(result_rows)
frontend

,flight_id,Дата вылета,Номер рейса,Аэропорт вылета,Аэропорт прилета,Время вылета,Время прилета,Время вылета (минуты),Время прилета (минуты),Тип ВС,...,Время в пути,Время вылета_ts,was_moved,stage,Доход,Емкость,Пассажиры,Бронирования,Загруженность,Доход на пассажира
0,1009_KGD_2024-08-01_03:00:00,2024-08-01,1009,KGD,SVO,03:00:00,06:35:00,180,395,32B,...,215.0,2024-08-01 03:00:00,False,result,8377.47,183,184,184,1.005464,45.529728
1,1009_KGD_2024-08-01_03:00:00,2024-08-01,1009,KGD,SVO,03:00:00,06:35:00,180,395,32B,...,215.0,2024-08-01 03:00:00,False,start,8377.47,183,184,184,1.005464,45.529728
2,1009_KGD_2024-08-02_03:00:00,2024-08-02,1009,KGD,SVO,03:00:00,06:35:00,180,395,32B,...,215.0,2024-08-02 03:00:00,False,result,8158.09,183,186,183,1.000000,43.860699
3,1009_KGD_2024-08-02_03:00:00,2024-08-02,1009,KGD,SVO,03:00:00,06:35:00,180,395,32B,...,215.0,2024-08-02 03:00:00,False,start,8158.09,183,186,183,1.000000,43.860699
4,1009_KGD_2024-08-03_03:00:00,2024-08-03,1009,KGD,SVO,03:00:00,06:35:00,180,395,32B,...,215.0,2024-08-03 03:00:00,False,result,10457.03,183,184,184,1.005464,56.831685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17217,851_SYX_2025-05-24_02:20:00,2025-05-24,851,SYX,KHV,02:20:00,10:15:00,140,615,32Q,...,475.0,2025-05-24 02:20:00,False,start,13252.25,184,78,76,0.413043,169.900641
17218,851_SYX_2025-05-27_02:20:00,2025-05-27,851,SYX,KHV,02:20:00,10:15:00,140,615,32Q,...,475.0,2025-05-27 02:20:00,False,result,14151.85,196,82,82,0.418367,172.583537
17219,851_SYX_2025-05-27_02:20:00,2025-05-27,851,SYX,KHV,02:20:00,10:15:00,140,615,32Q,...,475.0,2025-05-27 02:20:00,False,start,14151.85,196,82,82,0.418367,172.583537
17220,851_SYX_2025-05-31_02:20:00,2025-05-31,851,SYX,KHV,02:20:00,10:15:00,140,615,32Q,...,475.0,2025-05-31 02:20:00,False,result,17483.95,196,107,107,0.545918,163.401402


In [ ]:
frontend.to_json(
    "./frontend.json",
    force_ascii=False,
    orient="records"
)

In [185]:
# суммарный прогноз по рейсам на каждом этапе
revenue_by_flight: pd.DataFrame = df_combined.groupby(['Номер рейса', 'stage'])['Доход пасс'].sum().unstack(fill_value=0)

# разница дохода
revenue_by_flight['delta'] = revenue_by_flight['result'] - revenue_by_flight['start']
revenue_by_flight['pct_change'] = (revenue_by_flight['result'] - revenue_by_flight['start']) / revenue_by_flight['start'] * 100

# сортируем по приросту
revenue_by_flight = revenue_by_flight.sort_values('delta', ascending=False)

revenue_by_flight

stage,result,start,delta,pct_change
Номер рейса,,,,
209,2.643260e+07,26223909.95,208685.862023,0.795785
521,1.718629e+07,17009010.20,177284.190732,1.042296
275,1.947896e+07,19327070.78,151887.681934,0.785881
321,3.004922e+07,29912015.60,137202.732671,0.458688
1055,2.625188e+06,2522637.21,102550.410207,4.065206
1713,1.005205e+07,9958283.92,93770.631216,0.941634
1190,2.229170e+06,2137936.72,91232.818014,4.267330
1276,2.384685e+06,2300376.03,84308.976528,3.665008
1009,2.477701e+06,2393469.68,84231.037068,3.519202
